### We are going to build content based movie recomender

In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset can be found at:https://www.kaggle.com/datasets/rounakbanik/the-movies-dataset
About Dataset


Context

These files contain metadata for all 45,000 movies listed in the Full MovieLens Dataset. The dataset consists of movies released on or before July 2017. Data points include cast, crew, plot keywords, budget, revenue, posters, release dates, languages, production companies, countries, TMDB vote counts and vote averages.

This dataset also has files containing 26 million ratings from 270,000 users for all 45,000 movies. Ratings are on a scale of 1-5 and have been obtained from the official GroupLens website.
Content

This dataset consists of the following files:

movies_metadata.csv: The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

keywords.csv: Contains the movie plot keywords for our MovieLens movies. Available in the form of a stringified JSON Object.

credits.csv: Consists of Cast and Crew Information for all our movies. Available in the form of a stringified JSON Object.

links.csv: The file that contains the TMDB and IMDB IDs of all the movies featured in the Full MovieLens dataset.

links_small.csv: Contains the TMDB and IMDB IDs of a small subset of 9,000 movies of the Full Dataset.

ratings_small.csv: The subset of 100,000 ratings from 700 users on 9,000 movies.

The Full MovieLens Dataset consisting of 26 million ratings and 750,000 tag applications from 270,000 users on all the 45,000 movies in this dataset can be accessed here
Acknowledgements

This dataset is an ensemble of data collected from TMDB and GroupLens.
The Movie Details, Credits and Keywords have been collected from the TMDB Open API. This product uses the TMDb API but is not endorsed or certified by TMDb. Their API also provides access to data on many additional movies, actors and actresses, crew members, and TV shows. You can try it for yourself here.

The Movie Links and Ratings have been obtained from the Official GroupLens website. The files are a part of the dataset available here



## data preprocessing
cleaning the data

In [2]:
import ast
data=pd.read_csv('keywords.csv')
data


,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."
...,...,...
46414,439050,"[{'id': 10703, 'name': 'tragic love'}]"
46415,111109,"[{'id': 2679, 'name': 'artist'}, {'id': 14531,..."
46416,67758,[]
46417,227506,[]


In [3]:
data['keywords'] = data['keywords'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
data

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."
...,...,...
46414,439050,[tragic love]
46415,111109,"[artist, play, pinoy]"
46416,67758,[]
46417,227506,[]


In [4]:
kw_clean=data['keywords']
kw_clean

0        [jealousy, toy, boy, friendship, friends, riva...
1        [board game, disappearance, based on children'...
2        [fishing, best friend, duringcreditsstinger, o...
3        [based on novel, interracial relationship, sin...
4        [baby, midlife crisis, confidence, aging, daug...
                               ...                        
46414                                        [tragic love]
46415                                [artist, play, pinoy]
46416                                                   []
46417                                                   []
46418                                                   []
Name: keywords, Length: 46419, dtype: object

In [5]:
import ast
data3=pd.read_csv('movies_metadata.csv')
data3

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [6]:
data3['genres'] = data3['genres'].fillna('[]').apply(ast.literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
data3

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[Drama, Family]",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,[Drama],NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[Action, Drama, Thriller]",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [7]:
Movies_Data=data3[['title','genres','tagline','release_date','original_language']]
Movies_Data



,title,genres,tagline,release_date,original_language
0,Toy Story,"[Animation, Comedy, Family]",NaN,1995-10-30,en
1,Jumanji,"[Adventure, Fantasy, Family]",Roll the dice and unleash the excitement!,1995-12-15,en
2,Grumpier Old Men,"[Romance, Comedy]",Still Yelling. Still Fighting. Still Ready for...,1995-12-22,en
3,Waiting to Exhale,"[Comedy, Drama, Romance]",Friends are the people who let you be yourself...,1995-12-22,en
4,Father of the Bride Part II,[Comedy],Just When His World Is Back To Normal... He's ...,1995-02-10,en
...,...,...,...,...,...
45461,Subdue,"[Drama, Family]",Rising and falling between a man and woman,NaN,fa
45462,Century of Birthing,[Drama],NaN,2011-11-17,tl
45463,Betrayal,"[Action, Drama, Thriller]",A deadly game of wits.,2003-08-01,en
45464,Satan Triumphant,[],NaN,1917-10-21,en


Merging Data

In [8]:
Movies_Data['keywords']=kw_clean
Movies_Data


,title,genres,tagline,release_date,original_language,keywords
0,Toy Story,"[Animation, Comedy, Family]",NaN,1995-10-30,en,"[jealousy, toy, boy, friendship, friends, riva..."
1,Jumanji,"[Adventure, Fantasy, Family]",Roll the dice and unleash the excitement!,1995-12-15,en,"[board game, disappearance, based on children'..."
2,Grumpier Old Men,"[Romance, Comedy]",Still Yelling. Still Fighting. Still Ready for...,1995-12-22,en,"[fishing, best friend, duringcreditsstinger, o..."
3,Waiting to Exhale,"[Comedy, Drama, Romance]",Friends are the people who let you be yourself...,1995-12-22,en,"[based on novel, interracial relationship, sin..."
4,Father of the Bride Part II,[Comedy],Just When His World Is Back To Normal... He's ...,1995-02-10,en,"[baby, midlife crisis, confidence, aging, daug..."
...,...,...,...,...,...,...
45461,Subdue,"[Drama, Family]",Rising and falling between a man and woman,NaN,fa,[]
45462,Century of Birthing,[Drama],NaN,2011-11-17,tl,[]
45463,Betrayal,"[Action, Drama, Thriller]",A deadly game of wits.,2003-08-01,en,[based on novel]
45464,Satan Triumphant,[],NaN,1917-10-21,en,[woman director]


In [9]:
Movies_Data['original_language'].value_counts()

en    32269
fr     2438
it     1529
ja     1350
de     1080
      ...  
hy        1
gl        1
qu        1
uz        1
sm        1
Name: original_language, Length: 92, dtype: int64

Choosing the features that we want to recommend based on

In [10]:
features=['title','genres','keywords','tagline','original_language']
features

['title', 'genres', 'keywords', 'tagline', 'original_language']

In [11]:
for i in features:
    Movies_Data[i]=Movies_Data[i].fillna('')

In [12]:
Movies_Data=Movies_Data[['title','genres','keywords','tagline','original_language']]
Movies_Data.head()

,title,genres,keywords,tagline,original_language
0,Toy Story,"[Animation, Comedy, Family]","[jealousy, toy, boy, friendship, friends, riva...",,en
1,Jumanji,"[Adventure, Fantasy, Family]","[board game, disappearance, based on children'...",Roll the dice and unleash the excitement!,en
2,Grumpier Old Men,"[Romance, Comedy]","[fishing, best friend, duringcreditsstinger, o...",Still Yelling. Still Fighting. Still Ready for...,en
3,Waiting to Exhale,"[Comedy, Drama, Romance]","[based on novel, interracial relationship, sin...",Friends are the people who let you be yourself...,en
4,Father of the Bride Part II,[Comedy],"[baby, midlife crisis, confidence, aging, daug...",Just When His World Is Back To Normal... He's ...,en


In [13]:
Movies_Data.shape

(45466, 5)

Making a smaller subset due to limitited computing power.

In [28]:
Small_Movies_Data=Movies_Data.loc[0:7000]
Small_Movies_Data.shape

(7001, 5)

In [29]:
pre=str(Small_Movies_Data['genres'])+str(Small_Movies_Data['keywords'])+Small_Movies_Data['tagline']+Small_Movies_Data['original_language']
pre

0       0        [Animation, Comedy, Family]\n1       ...
1       0        [Animation, Comedy, Family]\n1       ...
2       0        [Animation, Comedy, Family]\n1       ...
3       0        [Animation, Comedy, Family]\n1       ...
4       0        [Animation, Comedy, Family]\n1       ...
                              ...                        
6996    0        [Animation, Comedy, Family]\n1       ...
6997    0        [Animation, Comedy, Family]\n1       ...
6998    0        [Animation, Comedy, Family]\n1       ...
6999    0        [Animation, Comedy, Family]\n1       ...
7000    0        [Animation, Comedy, Family]\n1       ...
Length: 7001, dtype: object

In [30]:
#convert the text data to vector
vectorizer=TfidfVectorizer()
vectorized=vectorizer.fit_transform(pre)
print(vectorized)

  (0, 3952)	0.18868265610717522
  (0, 2953)	0.07220087800934809
  (0, 6389)	0.07220087800934809
  (0, 1694)	0.07220087800934809
  (0, 3402)	0.07220087800934809
  (0, 4772)	0.07220087800934809
  (0, 1734)	0.07220087800934809
  (0, 3265)	0.07220087800934809
  (0, 4838)	0.07220087800934809
  (0, 589)	0.07220087800934809
  (0, 4837)	0.07220087800934809
  (0, 3164)	0.07220087800934809
  (0, 6077)	0.07220087800934809
  (0, 1204)	0.07220087800934809
  (0, 355)	0.07220087800934809
  (0, 3305)	0.07220087800934809
  (0, 1348)	0.07220087800934809
  (0, 5679)	0.07220087800934809
  (0, 1538)	0.07220087800934809
  (0, 6933)	0.07220087800934809
  (0, 3020)	0.07220087800934809
  (0, 1518)	0.07220087800934809
  (0, 198)	0.07220087800934809
  (0, 1371)	0.07220087800934809
  (0, 207)	0.07220087800934809
  :	:
  (7000, 2160)	0.05006928554000983
  (7000, 2161)	0.05006928554000983
  (7000, 698)	0.05006928554000983
  (7000, 6431)	0.05006928554000983
  (7000, 2878)	0.05006928554000983
  (7000, 3695)	0.0500692

Cosine Similarity

I will be using the Cosine Similarity to calculate a numeric quantity that denotes the similarity between two movies. 



In [31]:
Similarity=cosine_similarity(vectorized)
print(Similarity)

[[1.         0.58554034 0.51978756 ... 0.66536075 0.39904237 0.66878358]
 [0.58554034 1.         0.31951202 ... 0.4589852  0.26028066 0.43622293]
 [0.51978756 0.31951202 1.         ... 0.36306766 0.2177456  0.3649354 ]
 ...
 [0.66536075 0.4589852  0.36306766 ... 1.         0.29576191 0.4956885 ]
 [0.39904237 0.26028066 0.2177456  ... 0.29576191 1.         0.31425667]
 [0.66878358 0.43622293 0.3649354  ... 0.4956885  0.31425667 1.        ]]


In [32]:
Similarity.shape

(7001, 7001)

In [37]:
Name=input("Enter your favorite movie name please:")


Enter your favorite movie name please:jumanji


In [39]:
all_movies=Small_Movies_Data['title'].tolist()
all_movies

['Toy Story',
 'Jumanji',
 'Grumpier Old Men',
 'Waiting to Exhale',
 'Father of the Bride Part II',
 'Heat',
 'Sabrina',
 'Tom and Huck',
 'Sudden Death',
 'GoldenEye',
 'The American President',
 'Dracula: Dead and Loving It',
 'Balto',
 'Nixon',
 'Cutthroat Island',
 'Casino',
 'Sense and Sensibility',
 'Four Rooms',
 'Ace Ventura: When Nature Calls',
 'Money Train',
 'Get Shorty',
 'Copycat',
 'Assassins',
 'Powder',
 'Leaving Las Vegas',
 'Othello',
 'Now and Then',
 'Persuasion',
 'The City of Lost Children',
 'Shanghai Triad',
 'Dangerous Minds',
 'Twelve Monkeys',
 'Wings of Courage',
 'Babe',
 'Carrington',
 'Dead Man Walking',
 'Across the Sea of Time',
 'It Takes Two',
 'Clueless',
 'Cry, the Beloved Country',
 'Richard III',
 'Dead Presidents',
 'Restoration',
 'Mortal Kombat',
 'To Die For',
 'How To Make An American Quilt',
 'Se7en',
 'Pocahontas',
 'When Night Is Falling',
 'The Usual Suspects',
 'Guardian Angel',
 'Mighty Aphrodite',
 'Lamerica',
 'The Big Green',
 'Geo

In [40]:
#finding a close match to the givin movie
import difflib
find_best_match=difflib.get_close_matches(Name,all_movies)
print(find_best_match)


['Jumanji']


In [41]:
best_match=find_best_match[0]
best_match

'Jumanji'

In [42]:
#finding the index of the movie based on title
index_of_the_movie=Small_Movies_Data[Small_Movies_Data['title']==best_match].index.values[0]
index_of_the_movie



1

In [43]:
#getting the list of similar movies
similarity_score=list(enumerate(Similarity[index_of_the_movie]))
similarity_score


[(0, 0.5855403387685789),
 (1, 1.0000000000000002),
 (2, 0.3195120238975981),
 (3, 0.3379874577099095),
 (4, 0.3484948643620457),
 (5, 0.37298781629682226),
 (6, 0.36329345892382775),
 (7, 0.4422826160193416),
 (8, 0.4064669132239108),
 (9, 0.35746949500701714),
 (10, 0.37943988275361173),
 (11, 0.5855403387685789),
 (12, 0.37402371196121825),
 (13, 0.31890971155029874),
 (14, 0.2808508098217471),
 (15, 0.4226130285033003),
 (16, 0.3856460203229848),
 (17, 0.27069067196775476),
 (18, 0.36244354056520844),
 (19, 0.46955376249443836),
 (20, 0.35996636375194807),
 (21, 0.2654330897775355),
 (22, 0.36394419475818346),
 (23, 0.371352597496288),
 (24, 0.45785171347073317),
 (25, 0.45018707430898547),
 (26, 0.41585304936203),
 (27, 0.5855403387685789),
 (28, 0.36901910809927074),
 (29, 0.33040130334135276),
 (30, 0.43388521145598963),
 (31, 0.4860286039292901),
 (32, 0.5640504885533889),
 (33, 0.39296887829047045),
 (34, 0.3769316751360202),
 (35, 0.5855403387685789),
 (36, 0.5855403387685789

In [44]:
len(similarity_score)

7001

In [45]:
sorted_movies=sorted(similarity_score, key=lambda x:x[1], reverse=True)
sorted_movies


[(1, 1.0000000000000002),
 (0, 0.5855403387685789),
 (11, 0.5855403387685789),
 (27, 0.5855403387685789),
 (35, 0.5855403387685789),
 (36, 0.5855403387685789),
 (39, 0.5855403387685789),
 (42, 0.5855403387685789),
 (48, 0.5855403387685789),
 (51, 0.5855403387685789),
 (55, 0.5855403387685789),
 (59, 0.5855403387685789),
 (76, 0.5855403387685789),
 (82, 0.5855403387685789),
 (83, 0.5855403387685789),
 (85, 0.5855403387685789),
 (88, 0.5855403387685789),
 (89, 0.5855403387685789),
 (100, 0.5855403387685789),
 (107, 0.5855403387685789),
 (113, 0.5855403387685789),
 (115, 0.5855403387685789),
 (119, 0.5855403387685789),
 (128, 0.5855403387685789),
 (132, 0.5855403387685789),
 (134, 0.5855403387685789),
 (136, 0.5855403387685789),
 (137, 0.5855403387685789),
 (140, 0.5855403387685789),
 (145, 0.5855403387685789),
 (172, 0.5855403387685789),
 (175, 0.5855403387685789),
 (194, 0.5855403387685789),
 (197, 0.5855403387685789),
 (203, 0.5855403387685789),
 (218, 0.5855403387685789),
 (222, 0.585

In [46]:
print("Movies suggested for you are : \n")

i=1
for movies in sorted_movies :
    if (movies[1] <= 0.9):
        index=movies[0]
        title=Small_Movies_Data[Small_Movies_Data.index==index]['title'].values[0]
        if (i<11):
            print(i ,'.', title)
            i+=1

Movies suggested for you are : 

1 . Toy Story
2 . Dracula: Dead and Loving It
3 . Persuasion
4 . Dead Man Walking
5 . Across the Sea of Time
6 . Cry, the Beloved Country
7 . Restoration
8 . When Night Is Falling
9 . Mighty Aphrodite
10 . Kids of the Round Table
